In [ ]:
import cv2
import numpy as np

# Load the image containing both shoes
image = cv2.imread('TwoShoes.jpg')

# Resize the image to make it more manageable (optional)
image_resized = cv2.resize(image, (800, 600))

# Convert to grayscale
gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to remove noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply adaptive thresholding to detect regions with varying brightness
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 11, 2)

# Apply morphological transformations to close small gaps in the contours
kernel = np.ones((5, 5), np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Find contours using RETR_TREE to capture all contours
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Sort the contours by area to get the two largest (assuming two shoes)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]

# Check if two contours are found and highlight them visually
if len(contours) == 2:
    contour1 = contours[0]  # First shoe
    contour2 = contours[1]  # Second shoe
    
    # Compare shapes using matchShapes
    shape_similarity = cv2.matchShapes(contour1, contour2, cv2.CONTOURS_MATCH_I1, 0.0)
    
    # Compare sizes using contourArea
    area1 = cv2.contourArea(contour1)
    area2 = cv2.contourArea(contour2)
    size_difference = abs(area1 - area2)

    # Draw the contours for visualization
    cv2.drawContours(image_resized, [contour1], -1, (0, 255, 0), 2)  # Green for the first shoe
    cv2.drawContours(image_resized, [contour2], -1, (255, 0, 0), 2)  # Blue for the second shoe

    # Show the result with contours
    cv2.imshow("Shoes with Contours", image_resized)
    
    # Save the image with highlighted contours
    output_image_path = "/mnt/data/shoes_with_contours_adaptive_displayed.jpg"
    cv2.imwrite(output_image_path, image_resized)
    
    # Wait for any key to close the image window
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    result = {
        "shape_similarity": shape_similarity,
        "area1": area1,
        "area2": area2,
        "size_difference": size_difference,
        "output_image": output_image_path
    }
else:
    result = {
        "error": "Could not detect two shoes in the image."
    }

result
